In [2]:
import os 

from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file


groq_api_key=os.getenv("GROQ_API_KEY")

# print("GROQ_API_KEY:", groq_api_key)

# Initialzing the Groq model 

In [5]:
from langchain_groq import ChatGroq

model=ChatGroq(
    model="Gemma2-9b-It",
    groq_api_key=groq_api_key
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000230175CDFC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000230175CF220>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

humanmessage="Hi my name is brahmesh,im an AI engineer"

response=model.invoke([
    # SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=humanmessage)
])


In [13]:
response.content

"Hi Brahmesh, it's nice to meet you!  \n\nThat's awesome that you're an AI engineer. It's a really exciting field right now. \n\nWhat kind of AI work do you do?  Do you have any projects you're particularly proud of?\n"

In [14]:
from langchain_core.messages import AIMessage,HumanMessage

model.invoke([
    HumanMessage(content=humanmessage),
    AIMessage(content="Hi Brahmesh, it's nice to meet you!  \n\nThat's awesome that you're an AI engineer. It's a really exciting field right now. \n\nWhat kind of AI work do you do?  Do you have any projects you're particularly proud of?\n"),
    HumanMessage(content="Hey what is my name and what do i do")
])

AIMessage(content="You told me your name is Brahmesh, and you said you are an AI engineer!  \n\nIs there anything else you'd like to tell me about yourself? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 101, 'total_tokens': 140, 'completion_time': 0.072849637, 'prompt_time': 0.007743982, 'queue_time': 0.39306899700000003, 'total_time': 0.080593619}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ef03211-61f5-456f-8567-0e40b299e4e7-0', usage_metadata={'input_tokens': 101, 'output_tokens': 39, 'total_tokens': 140})

# Message History

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from  langchain_core.runnables.history import RunnableWithMessageHistory


# while user is using the model,we should make sure that one 
# session is different from another


store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()

    return store[session_id]

# # Enables the model to remember previous messages using session history
with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [18]:
config={"configurable":{"session_id":"chat1"}}

In [20]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is brahmesh,im an AI engineer")],
    config=config
)

In [21]:
response.content

"Hi Brahmesh, it's great to meet you! \n\nBeing an AI engineer is such a cool field. What kind of projects are you working on? Are you into computer vision, natural language processing, or something else entirely?  \n\nI'm always interested in hearing about what people are building with AI.  \n\n"

In [22]:
# Playing with different sessions,wheather it is going to remember or not
# 1) giving same session id
config1={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hey what is my name")],
    config=config1
)

response.content

'Your name is Brahmesh 😊 \n\nI remember you told me when we first met!  \n'

In [23]:
# Now we are going to give different session id
config2={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hey what is my name")],
    config=config2
)

response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

## I can conclude that to preserve the chat history,make sure you maintain same session id

# Prompt template

In [25]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt | model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is brahmesh")]})

AIMessage(content="Hi Brahmesh, it's nice to meet you!\n\nWhat can I do to help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 21, 'total_tokens': 49, 'completion_time': 0.050909091, 'prompt_time': 0.003462605, 'queue_time': 0.24607632399999999, 'total_time': 0.054371696}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--51f61651-e601-4351-925a-19a4d7883b8b-0', usage_metadata={'input_tokens': 21, 'output_tokens': 28, 'total_tokens': 49})

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)


In [29]:
config={"configurable":{"session_id":"chat3"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is brahmesh")],
    config=config
)

response.content

"Hi Brahmesh, it's nice to meet you!\n\nIs there anything I can help you with today? \n\nAs an AI assistant, I can help with a variety of tasks, such as:\n\n* **Generating creative content:**  I can write stories, poems, articles, and even code.\n* **Answering your questions:** I have access to a vast store of knowledge and can answer your questions on a wide range of topics.\n* **Summarizing text:**  I can condense large amounts of text into shorter summaries.\n* **Translating languages:** I can translate text between many different languages.\n\n\nLet me know how I can be of service!\n"

In [30]:
config={"configurable":{"session_id":"chat3"}}

response=with_message_history.invoke(
    [HumanMessage(content="heyy what is my name")],
    config=config
)

response.content

'Your name is Brahmesh!  😊 \n\nI remember from our previous conversation.  Is there anything else I can help you with?  \n\n\n\n'

In [31]:
# Add more complexity 

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant.Answer in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt | model

In [32]:
response=chain.invoke(
    {
        "messages":[HumanMessage(content="Hi my name is brahmesh")],
        "language":"Hindi"
    }
)

response.content

'Namaste Brahmesh! 😊  मैं आपकी मदद करने के लिए यहाँ हूँ। आप क्या जानना चाहते हैं या क्या करने में मेरी मदद चाहिए? \n'

In [33]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Hi my name is brahmesh")],
        "language":"Hindi"
    },
    config=config
)

response.content

'नमस्ते ब्रह्मेश! 😊 \n\nमुझे बहुत खुशी है तुम्हारी मुलाक़ात करने में। मैं तुम्हारी मदद करने के लिए तैयार हूँ।  आप क्या जानना चाहेंगे? \n'

In [35]:
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Heyy what is my name")],
        "language":"Hindi"
    },
    config=config
)

response.content

'तुम्हारा नाम ब्रह्मेश है! 😊  \n'

# How to manage the converstion history

## trim_messages is a function that shortens the chat history by keeping only the most important or recent messages so the conversation fits within the model’s limits.

In [40]:
#!pip install transformers
import warnings
warnings.filterwarnings("ignore")

In [43]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi my name is brahmesh"),
    AIMessage(content="Hi Brahmesh, it's nice to meet you!  \n\nThat's awesome that you're an AI engineer. It's a really exciting field right now. \n\nWhat kind of AI work do you do?  Do you have any projects you're particularly proud of?\n"),
    HumanMessage(content="Hey what is my name and what do i do"),
    AIMessage(content="Your name is Brahmesh and you are an AI engineer."),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="2+2=4"),
    HumanMessage(content="what is 2+3"),
    AIMessage(content="2+3=5"),
    HumanMessage(content="enjoy talking to you"),
    AIMessage(content="I enjoy talking to you too!"),


]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hey what is my name and what do i do', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Brahmesh and you are an AI engineer.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2+2=4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2+3=5', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='enjoy talking to you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I enjoy talking to you too!', additional_kwargs={}, response_metadata={})]

In [44]:
# now trim the tokens to 45
# and see how the model is going to respond
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi my name is brahmesh"),
    AIMessage(content="Hi Brahmesh, it's nice to meet you!  \n\nThat's awesome that you're an AI engineer. It's a really exciting field right now. \n\nWhat kind of AI work do you do?  Do you have any projects you're particularly proud of?\n"),
    HumanMessage(content="Hey what is my name and what do i do"),
    AIMessage(content="Your name is Brahmesh and you are an AI engineer."),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="2+2=4"),
    HumanMessage(content="what is 2+3"),
    AIMessage(content="2+3=5"),
    HumanMessage(content="enjoy talking to you"),
    AIMessage(content="I enjoy talking to you too!"),


]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2+3=5', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='enjoy talking to you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I enjoy talking to you too!', additional_kwargs={}, response_metadata={})]

In [46]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")| trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {"messages":messages + [HumanMessage(content="what is my name")],
        "language":"English"},
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n\n"

In [48]:
# We got that message because we are using the trimmer and cut the maxtokens to 45
# and it is not allowing the model to see the previous messages

In [49]:
response=chain.invoke(
    {"messages":messages + [HumanMessage(content="what math problem did i ask")],
        "language":"English"},
)

response.content

'You asked:  "What is 2+3?" \n'

In [61]:
## Lets wrap this in the message history

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [62]:
response=with_message_history.invoke(
    {"messages":messages + [HumanMessage(content="what is my name")],
        "language":"English"},
    config=config
)

response.content

"As an AI, I have no memory of past conversations and don't know your name. Would you like to tell me? 😊  \n"

In [64]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="what math problem did i ask")],
        "language":"English"},
    config=config
)

response.content

"As a helpful assistant with no memory of past conversations, I have no record of any math problems you may have asked before. \n\nIf you have a math problem you'd like help with, please ask! I'm happy to assist. 😊  \n\n"